In [1]:
import sys
%cd ..
sys.path.append('.')

/home/oleg/yodas_conversion


In [2]:
from pathlib import Path
import pickle
from typing import Literal, Any
from dataclasses import dataclass

import numpy as np
import pandas as pd
from datasets import load_from_disk, Dataset, concatenate_datasets
from tqdm.auto import tqdm
from scipy.special import log_softmax, expit
import IPython.display
import matplotlib.pyplot as plt

from src.spectrogram_transformer import AudioSpectrogramTransformer, SEGMENT_SHIFT, SEGMENT_LENGTH
from src.yodas_search import YodasSearch, YodasSearchResult
from src.audioset_utils import AudioSetOnthology, sliding_window_mean, Features
from src.speaker_diarization import Segment, get_speech_mask, reorder_speakers

In [3]:
yodas2 = concatenate_datasets([
    (
        concatenate_datasets([load_from_disk(path) for path in tqdm(sorted(Path(dataset_path).glob('*-of-*')))])
        .map(lambda s: {'audio': {'path': dataset_path + '/' + s['audio']['path']}})
    ) #.take(100)
    for dataset_path in [
        'datasets/yodas2_ru000_32k',
        'datasets/yodas2_ru001_32k'
    ]
])

# yodas2 = load_from_disk('datasets/yodas2_ru000_32k/00000-of-00500')

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/287 [00:00<?, ?it/s]

In [4]:
searcher = YodasSearch(
    dataset=yodas2,
    onthology=AudioSetOnthology(),
    cache_file='tmp/search_features.pkl',
    # rewrite_cache=True
)

In [5]:
searcher.print_features()

Features:
  text
    ndarray(shape=(52553,), dtype=object, obj_type=<class 'str'>)
  video_id
    ndarray(shape=(52553,), dtype=<U11)
  duration
    ndarray(shape=(52553,), dtype=float32)
  speaker_embeddings
    ndarray(shape=(52553,), dtype=object, obj_type=<class 'numpy.ndarray'>)
  is_music
    Features(52553 samples, shape (1194474, 155), bool, 177 MB, start 0 sec, delta 20 sec, length 120 sec)
  acoustic
    Features(52553 samples, shape (1194474, 366), bool, 417 MB, start 0 sec, delta 20 sec, length 120 sec)
  warnings
    Features(52553 samples, shape (1194474, 21), bool, 24 MB, start 0 sec, delta 20 sec, length 120 sec)
  for_manual_search
    Features(52553 samples, shape (1194474, 3), float32, 14 MB, start 0 sec, delta 20 sec, length 120 sec)
  speech_seconds
    Features(52553 samples, shape (1194474, 1), float32, 5 MB, start 0 sec, delta 20 sec, length 120 sec)
  voice_overlap_seconds
    Features(52553 samples, shape (1194474, 1), float32, 5 MB, start 0 sec, delta 20 sec,

In [ ]:
searcher._features['is_music'].reduce_by_feature_axis('max')

Features(52553 samples, shape (1194474, 1), bool, 1 MB, start 0 sec, delta 20 sec, length 120 sec)